# Meander Maker

If you haven't ever used the `googlemaps` python package, there are a few small setup steps to take first. You'll need to install it by running the next cell. Additionally, we will use `gmplot` to render the outputs. Finally, generate an API key to interface with Google's services. (There is a free tier, but if you use it a lot, they will start charging you for access.)

In [35]:
!pip install googlemaps
!pip install gmplot

    100% |████████████████████████████████| 122kB 8.3MB/s 
  Stored in directory: /Users/dakaspar/Library/Caches/pip/wheels/81/6a/76/4dd6a7cc310ba765894159ee84871e8cd55221d82ef14b81a1
Successfully built gmplot


If you would like to tinker around with the code, you'll need your own API key for Google Maps. You can register one from this address: [https://developers.google.com/places/web-service/get-api-key](https://developers.google.com/places/web-service/get-api-key). Once you have one, create a file in the directory containing this notebook called `.secret.key` and simply paste in your newly generated API key, no quotes or line breaks or anything. Save, exit, and you should be good to go.

In [1]:
import pandas as pd
from datetime import datetime
import googlemaps
from gmplot import gmplot

with open('.secret.key') as f:
    api_key = f.read().strip()

gmaps = googlemaps.Client(key=api_key)

Let's decide which topic to explore:

In [70]:
def get_loc(current=True):
    """
    current = True: Attempt to determine the user location 
    current = False: Ask the user for a starting location.
    """
    output=None
    if current is True:
        output = gmaps.geolocate()['location']
    else:
        query = input('Where would you like to start?')
        if query.count(',') == 1 and query.translate({ord(i):None for i in '- ,:.'}).isnumeric():
            output = {}
            output['lat'], output['lng'] = [float(x) for x in query.split(',')]
        else:
            place_id = gmaps.find_place(query, input_type='textquery')['candidates'][0]['place_id']
            output = gmaps.place(place_id)['result']['geometry']['location']
    return output

In [71]:
get_loc(current=False)

Where would you like to start? 47.6798338, -122.3257826


{'lat': 47.6798338, 'lng': -122.3257826}

In [28]:
keyword = input('What theme walk would you like to explore today?')

What theme walk would you like to explore today? bookstore


In [29]:
temp = gmaps.places_nearby((47.6225507, -122.5167143), radius=3000, keyword=keyword)

In [30]:
[x['name'] for x in temp['results']]

['Eagle Harbor Book Co', 'Backstreet Beat', 'Christian Science Reading Room']

In [ ]:
def nearify(start=None, keyword=keyword):
    """
    Used to avoid multiple locations with the same name. This will auto-search 
    for the closest matches to the starting location, even when other (potentially more 
    famous) locations exist further away.
    """
    if start is None:
        start = dest_list[0]
    
    gmaps.places_nearby()
    

In [8]:
dest_list = ['Genoa Cellars', 'Gorman -Old Scratch Winery', 'Long Cellars', 'LOVE THAT RED', 'Ancestry Cellars']

def round_trip(circle=True, dest_list=dest_list):
    """Append the Start location to the end of the list, resulting in a loop."""
    return dest_list.extend(dest_list[0])

def walk(dest_list, verbose=False):
    """Given a list of places to visit, return the walking dist & time."""

    if len(dest_list) > 10:
        return "There is a maximum of 10 stops per adventure."
    
    #assert len(dest_list) < 11, "There is a maximum of 10 stops per adventure."
    
    start, stop = dest_list[0], dest_list[-1]
    
    waypoints = None
    if len(dest_list) > 2:
        waypoints = dest_list[1:-1]
    
    directions_result = gmaps.directions(
        start, stop, mode="walking", waypoints=waypoints, optimize_waypoints=True
    )
    
    if verbose:
        dist = 0
        time = 0
        for leg in directions_result[0]['legs']:
            td = leg['distance']['value']
            tt = leg['duration']['value']
            print(f'+{td} m --and-- +{round(tt/60, 2)} min')
            dist += td
            time += tt
        print(f'total dist: {dist} m \nest time: {round(time / 60, 1)} min')
    return directions_result

In [31]:
# path = walk(dest_list)
# path

In [33]:
path[0]['bounds']

{'northeast': {'lat': 47.8851198, 'lng': -119.9709734},
 'southwest': {'lat': 47.17528739999999, 'lng': -122.1541535}}

In [65]:
gmaps.find_place('LOVE THAT RED', input_type='textquery', location_bias='point:47.769302, -122.148902')

{'candidates': [{'place_id': 'ChIJ1c9qHpgOkFQRPaH6VxO7Lhs'}], 'status': 'OK'}

In [66]:
gmaps.place('ChIJ1c9qHpgOkFQRPaH6VxO7Lhs')

{'html_attributions': [],
 'result': {'address_components': [{'long_name': 'D100',
    'short_name': 'D100',
    'types': ['subpremise']},
   {'long_name': '19501', 'short_name': '19501', 'types': ['street_number']},
   {'long_name': '144th Avenue Northeast',
    'short_name': '144th Ave NE',
    'types': ['route']},
   {'long_name': 'North Industrial',
    'short_name': 'North Industrial',
    'types': ['neighborhood', 'political']},
   {'long_name': 'Woodinville',
    'short_name': 'Woodinville',
    'types': ['locality', 'political']},
   {'long_name': 'King County',
    'short_name': 'King County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Washington',
    'short_name': 'WA',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '98072', 'short_name': '98072', 'types': ['postal_code']}],
  'adr_address': 'Back Entrance, <span c

In [106]:
ltr_gc = gmaps.directions('Warr-King Wines', 'Sol Stone Winery', mode='walking', waypoints=['Efeste'])

In [69]:
gmapit = gmplot.GoogleMapPlotter(47.7690488, -122.1514703, zoom=15, apikey=api_key)

In [59]:
gmapit.plot([47.7690488], [-122.1514703], 'cornflowerblue', edge_width=10)
gmapit.scatter([47.7690488], [-122.1514703], '#3B0B39', size=40, marker=False)
gmapit.scatter([47.7690488], [-122.1514703], 'k', marker=True)
#gmap.heatmap(heat_lats, heat_lngs)

In [60]:
gmapit.draw("mymap.html")

In [99]:
loc = gmaps.geolocate()['location']

In [121]:
query = input('Where would you like to start?')
place = gmaps.find_place(query, input_type='textquery')

Where would you like to start? Cornuto Pizzaria


In [122]:
place

{'candidates': [{'place_id': 'ChIJjXPLOTsUkFQRePof7WFiGuc'}], 'status': 'OK'}

In [124]:
temp = gmaps.place(place['candidates'][0]['place_id'])
temp[]

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '7404',
    'short_name': '7404',
    'types': ['street_number']},
   {'long_name': 'Greenwood Avenue North',
    'short_name': 'Greenwood Ave N',
    'types': ['route']},
   {'long_name': 'Northwest Seattle',
    'short_name': 'Northwest Seattle',
    'types': ['neighborhood', 'political']},
   {'long_name': 'Seattle',
    'short_name': 'Seattle',
    'types': ['locality', 'political']},
   {'long_name': 'King County',
    'short_name': 'King County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Washington',
    'short_name': 'WA',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '98103', 'short_name': '98103', 'types': ['postal_code']},
   {'long_name': '5044',
    'short_name': '5044',
    'types': ['postal_code_suffix']}],
  'adr_address': '<span class

In [102]:
gmapit.plot([loc['lat']], [loc['lng']])
gmapit.draw("mymap.html")

In [110]:
ltr_gc[0]['legs']

[{'distance': {'text': '0.4 mi', 'value': 608},
  'duration': {'text': '9 mins', 'value': 516},
  'end_address': '19730 144th Ave NE, Woodinville, WA 98072, United States',
  'end_location': {'lat': 47.7711178, 'lng': -122.146107},
  'start_address': '19501 144th Ave NE d700, Woodinville, WA 98072, USA',
  'start_location': {'lat': 47.7693978, 'lng': -122.1514714},
  'steps': [{'distance': {'text': '118 ft', 'value': 36},
    'duration': {'text': '1 min', 'value': 28},
    'end_location': {'lat': 47.7697239, 'lng': -122.1514725},
    'html_instructions': 'Head <b>north</b> toward <b>144th Ave NE</b>',
    'polyline': {'points': 'w}pbHtuphV_A?'},
    'start_location': {'lat': 47.7693978, 'lng': -122.1514714},
    'travel_mode': 'WALKING'},
   {'distance': {'text': '0.2 mi', 'value': 279},
    'duration': {'text': '4 mins', 'value': 224},
    'end_location': {'lat': 47.76962109999999, 'lng': -122.1477658},
    'html_instructions': 'Turn <b>right</b>',
    'maneuver': 'turn-right',
    'p

In [111]:
ltr_gc[0]

{'bounds': {'northeast': {'lat': 47.7711178, 'lng': -122.146107},
  'southwest': {'lat': 47.7662737, 'lng': -122.1514725}},
 'copyrights': 'Map data ©2019',
 'legs': [{'distance': {'text': '0.4 mi', 'value': 608},
   'duration': {'text': '9 mins', 'value': 516},
   'end_address': '19730 144th Ave NE, Woodinville, WA 98072, United States',
   'end_location': {'lat': 47.7711178, 'lng': -122.146107},
   'start_address': '19501 144th Ave NE d700, Woodinville, WA 98072, USA',
   'start_location': {'lat': 47.7693978, 'lng': -122.1514714},
   'steps': [{'distance': {'text': '118 ft', 'value': 36},
     'duration': {'text': '1 min', 'value': 28},
     'end_location': {'lat': 47.7697239, 'lng': -122.1514725},
     'html_instructions': 'Head <b>north</b> toward <b>144th Ave NE</b>',
     'polyline': {'points': 'w}pbHtuphV_A?'},
     'start_location': {'lat': 47.7693978, 'lng': -122.1514714},
     'travel_mode': 'WALKING'},
    {'distance': {'text': '0.2 mi', 'value': 279},
     'duration': {'tex

In [125]:
import json

In [126]:
json.scanner()

AttributeError: 'dict' object has no attribute 'read'